In [1]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
book_id = "15881"
url = "https://www.goodreads.com/book/show/" + book_id
source = urlopen(url, timeout= 6000)
soup = BeautifulSoup(source, "html.parser")

In [ ]:
soup

In [4]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <script>\n   var ue_t0=window.ue_t0||+new Date();(function(e){var c=e,a={main_scope:"mainscopecsm",q:[],t0:c.ue_t0||+new Date(),d:g};function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(k,j,h){var i={m:k,f:j,l:h,fromOnError:1,args:arguments};c.ueLogError(i);return false}b.skipTrace=1;e.onerror=b;function f(){c.uex("ld")}if(e.addEventListener){e.addEventListener("load",f,false)}else{if(e.attachEvent){e.attachEvent("onload",f)}}a.tag=d("tag");a.log=d("log");a.reset=d("rst");c.ue_csm=c;c.ue=a;c.ueLogError=d("err");c.ues=d("ues");c.uet=d("uet");c.uex=d("uex");c.uet("ue")})(window);(function(e,d){var a=e.ue||{};function c(g){if(!g){return}var f=d.head||d.getElementsByTagName("head")[0]||d.documentElement,h=d.createElement("script");h.async="async";h.src=g;f.insertBefore(h,f.firstChild)}function b(){var k=e.ue_cdn||"z-ecx.images-amazon.com",g=e.ue_cdns||"images-na.ssl-images-

In [3]:
def get_description(soup):
    return soup.find("div", {"data-testid": "description"}).findAll("span")[-1].text

In [6]:
# abc = get_description(soup)

In [8]:
abc = soup.find("img", {"class": "ResponsiveImage"}).attrs.get("src")

In [9]:
abc

'https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1474169725i/15881.jpg'

In [10]:
def get_series_name(soup):
    series = soup.find(id="bookSeries").find("a")
    if series:
        series_name = re.search(r"\((.*?)\)", series.text).group(1)
        return series_name
    else:
        return None

In [43]:
num_ratings = soup.find("span", {"data-testid": "ratingsCount"}).text

In [44]:
num_ratings

'3,880,823\xa0ratings'

In [45]:
num_ratings = re.search(r"(.*)\s", num_ratings)[0].strip().replace(",", "")
int(num_ratings)

3880823

In [46]:
def get_num_ratings(soup):

    num_ratings = soup.find("span", {"data-testid": "ratingsCount"}).text
    num_ratings = re.search(r"(.*)\s", num_ratings)[0].strip().replace(",", "")
    
    return int(num_ratings)


In [48]:
def get_num_reviews(soup):

    num_reviews = soup.find("span", {"data-testid": "reviewsCount"}).text
    num_reviews = re.search(r"(.*)\s", num_reviews)[0].strip().replace(",", "")
    
    return int(num_reviews)    

In [60]:
def get_avg_ratings(soup):

    avg_ratings = soup.find("div", {"class": "RatingStatistics__rating"}).text
    
    return float(avg_ratings)  

In [49]:
get_num_reviews(soup)

78444

In [61]:
get_avg_ratings(soup)

4.43

In [62]:
def get_author_id(soup):
    author_url = soup.find("a", {"class": "Avatar Avatar--large"}).attrs.get("href")
    return author_url.split("/")[-1]

In [63]:
get_author_id(soup)

'1077326.J_K_Rowling'

In [66]:
def get_genres(soup):
    genres = []
    for node in soup.find_all("span", {"class": "BookPageMetadataSection__genreButton"}):
        current_genres = node.find_all(
            "a", {"class": "Button Button--tag-inline Button--small"}
        )
        current_genre = " > ".join([g.text for g in current_genres])
        if current_genre.strip():
            genres.append(current_genre)
    return genres

In [67]:
get_genres(soup)

['Fantasy',
 'Fiction',
 'Young Adult',
 'Magic',
 'Childrens',
 'Middle Grade',
 'Audiobook']

In [5]:
def get_rating_distribution(soup):

    all_rating = soup.find_all("div", {"data-testid": re.compile(r'labelTotal')})
    distribution_dict = {}
    for rating in all_rating:

        rate = int(re.search(r"\d", rating.attrs.get("data-testid")).group())

        num_rating = int(re.search(r"(.*)\s", rating.text)[0].strip().replace(",", ""))

        distribution_dict[rate] = num_rating

    return distribution_dict


In [14]:
all_rating = soup.find_all("div", {"data-testid": re.compile(r'labelTotal')})
all_rating

[<div aria-label="Number of ratings and percentage of total ratings" class="RatingsHistogram__labelTotal" data-testid="labelTotal-5">2,303,676 (59%)</div>,
 <div aria-label="Number of ratings and percentage of total ratings" class="RatingsHistogram__labelTotal" data-testid="labelTotal-4">1,065,509 (27%)</div>,
 <div aria-label="Number of ratings and percentage of total ratings" class="RatingsHistogram__labelTotal" data-testid="labelTotal-3">419,681 (10%)</div>,
 <div aria-label="Number of ratings and percentage of total ratings" class="RatingsHistogram__labelTotal" data-testid="labelTotal-2">71,863 (1%)</div>,
 <div aria-label="Number of ratings and percentage of total ratings" class="RatingsHistogram__labelTotal" data-testid="labelTotal-1">23,162 (&lt;1%)</div>]

In [30]:
distribution_dict = {}
for rating in all_rating:

    rate = int(re.search(r"\d", rating.attrs.get("data-testid")).group())

    num_rating = int(re.search(r"(.*)\s", rating.text)[0].strip().replace(",", ""))

    distribution_dict[rate] = num_rating

    


In [31]:
distribution_dict

{5: 2303676, 4: 1065509, 3: 419681, 2: 71863, 1: 23162}